<p style="text-align: center; font-weight: bold; font-size: 16pt;font-family: "Computer Modern", sans-serif">Problem Set 2 </p> 
<p style="text-align: right; font-size: 12pt; font-family: "Computer Modern", sans-serif">Erik Butcher</p>

#### Q.1 
\begin{align}
E = E^o + \frac{RT}{nF}ln\Bigl(\frac{[Ox]}{[R]}\Bigr)= E^o + \frac{0.059}{n}log_{10}\Bigl(\frac{[Ox]}{[R]}\Bigr) \\ \; \\
Fe^{3+} + e^- \rightarrow Fe^{2+} & \Leftarrow \boxed{ \texttt{Gain of electron - Reduction}} \\ \; \\
[Re] + e^- \rightarrow [Ox] \\ \; \\
Answer \;(both \; valid): \\
\boldsymbol{ c) \;\; E = E^o + \frac{0.059}{n}ln\Bigl(\frac{[Fe^{3+}]}{[Fe^{2+}]}\Bigr)} \\ \; \\
\boldsymbol{ d) \;\; E = E^o - \frac{0.059}{n}ln\Bigl(\frac{[Fe^{2+}]}{[Fe^{3+}]}\Bigr)} 
\end{align}

#### Q.2
\begin{align*}
E_{probe} = K + 0.059\cdot log[F^-] \\ \; \\
E_{probe} = V\cdot q & \Leftarrow \boxed{ \texttt{charge ratiometric, so regarded q as 1}} \\ \; \\
V_0 = K + 0.059\cdot log[F^-] \\ \; \\
0.112V = K + 0.059\cdot log[0.01] \\ \; \\
0.112V = K - 0.059 & \Leftarrow \boxed{ \texttt{K = 0.23V}} \\ \; \\
V_t = K + 0.059\cdot log[F^-_t] \\ \; \\ 
0.203V = 0.23V + 0.059\cdot log[F^-_t] \\ \; \\
-0.027 = 0.059\cdot log[F^-_t] \\ \; \\
Answer: \\
\boldsymbol{[F^-_t] = 0.35M}
\end{align*} 

#### Q.3

In [94]:
import numpy as np;
import pylab as plt;
from scipy.integrate import odeint;

# conversion constants
nA = 1e9; us = 1e-6; MOhms = 1e6; pF = 1e-12; mV = 1e-3; mS = 1e-3

In [ ]:
dt = 10e-6 # 10 us
t_max = 1 # seconds
time = np.arange(0, t_max, dt)
R_m = 50e6 # 50 MOhms Membrane Resistance
C_m = 200e-12 # 200 pF Membrane capacitance
V_e = -70e-3 # -70 mV Equilibrium voltage
V_reset = -80e-3 # -80 mV Hyperpolarization/reset voltage
V_th = -40e-3 # -40 mV Threshold voltage
Input_Is = np.asarray([100e-12, 1e-9, 10e-9]) # Input currents

def voltageModel(dt, R_m, C_m, V_e, V_reset, V_th, Input_Is):
    V_m = np.zeros(time.shape)
    V_m[0] = V_e
   
    for i in range(1, V_m.shape[0]):   
        dV = dt * (-1/R_m * (V_m[i-1]-V_e) + Input_Is)/C_m
        V_m[i] = V_m[i-1] + dV
        
        if V_m[i-1] >= V_th:
            V_m[i] = V_reset
            
    return V_m

plt.figure(1, figsize=(15,10))
for I in range(Input_Is.shape[0]):
    plt.subplot(3, 1, I+1)
    plt.plot(time, voltageModel(dt, R_m, C_m, V_e, V_reset, V_th, Input_Is[I]))
    plt.ylabel('V')
    plt.xticks(np.arange(0, t_max, 0.005))
    
plt.xlim((0,0.05))

plt.xlabel('time (x)')
plt.ylabel('x')

Text(0, 0.5, 'x')

#### Q3 - Part 2

In [ ]:
T_ref = 3e-3 # 3 mS Refractory time
R_m = 50e6 # 50 MOhms Membrane Resistance
C_m = 200e-12 # 200 pF Membrane capacitance
V_e = -70e-3 # -70 mV Equilibrium voltage
V_reset = -80e-3 # -80 mV Hyperpolarization/reset voltage
V_th = -40e-3 # -40 mV Voltage threshold

def frequencyModel(T_ref, R_m, C_m, V_e, V_reset, V_th, I):
    Ith = (V_th - V_e)/R_m
    if I < Ith:
        return 0
    
    f = 1/(T_ref + R_m*C_m*np.log((I*R_m+V_e-V_reset)/(I*R_m+V_e-V_th)))
    
    return f

I = np.arange(0, 10e-9, 100e-12)
f_calc = np.zeros(I.shape[0])

nAf = lambda a: a*nA

for i in range(f_calc.shape[0]):
    f_calc[i] = frequencyModel(T_ref, R_m, C_m, V_e, V_reset, V_th, I[i])

plt.plot(I*nA,f_calc) # convert to nanoamps
plt.xlabel("I (nA)")

In [ ]:
dt = 10e-6
t_max = 1
time = np.arange(0, t_max, dt)

def spikeIFModel(dt, R_m, C_m, V_e, V_reset, V_th, Input_Is):
    V_m = np.zeros(time.shape)
    Spike_count = 0 
    V_m[0] = V_e
    # Let's figure out dV
    for i in range(1, V_m.shape[0]):   #[0] length of it
        dV = dt * (-1/R_m * (V_m[i-1]-V_e) + Input_Is)/C_m
        V_m[i] = V_m[i-1] + dV
       
        if V_m[i-1] >= V_th:
            V_m[i] = V_reset
            Spike_count += 1

    return Spike_count

f_sim = np.zeros(I.shape[0])
for i in range(I.shape[0]):
    f_sim[i] = spikeIFModel(dt, R_m, C_m, V_e, V_reset, V_th, I[i])

plt.plot(I*1e9, f_calc)
plt.plot(I*1e9, f_sim)
plt.xlabel("I (nA)")

In [ ]:
# Part 4
dt = 10e-6
t_max = 10
time = np.arange(0, t_max, dt)
T_ref = 3e-3 # Refactory time
R_m = 50e6 # 50 MOhms Membrane Resistance
C_m = 200e-12 # 200 pF Membrane capacitance
V_e = -70e-3 # -70 mV Equilibrium voltage
V_reset = -80e-3 # -80 mV Hyperpolarization/reset voltage
V_th = -40e-3

def gaussSpikeIFModel(dt, R_m, C_m, V_e, V_reset, V_th, Isigma):
    V_m = np.zeros(time.shape)
    Imean = 1000e-12
    Inoisy = np.random.normal(Imean, Isigma, (time.shape[0], 1))
    Spike_count = 0
    V_m[0] = V_e 
    #Let's figure out d
    spikes = []
    last_spike_idx = 0    
    for i in range(1, V_m.shape[0]):
        dV = dt* (-1/R_m * (V_m[i-1]-V_e) + Inoisy[i])/C_m
        V_m[i] = V_m[i-1] + dV
        
        if V_m[i-1] >= V_th:
            V_m[i] = V_reset
            spikes.append((i-last_spike_idx)*dt) # times dt to get actual time in ms
            last_spike_idx = i
            
    return np.asarray(spikes) # inter-spike periods
    
Input_Is = np.asarray([0, 100e-12, 500e-12])

plt.figure(1, figsize=(15,10))

for I in range(Input_Is.shape[0]):
    plt.subplot(3, 1, I+1)
    spikecounts = gaussSpikeIFModel(dt, R_m, C_m, V_e, V_reset, V_th, Input_Is[I])
    print(spikecounts.shape[0])
    plt.hist(spikecounts, bins=50, range=(0.010, 0.012))
    # plt.title("I=" + str(Input_Is[I]*1e9) + "nA")
    plt.ylabel("Number")

plt.xlabel("Inter - Spike Period")
    

In [ ]:
Alpha_n = lambda V_m:0.01*(V_m+55)/(1-np.exp(-(V_m+55)/10))
Alpha_m = lambda V_m: 0.1*(V_m+40)/(1-np.exp(-(V_m+40)/10))
Alpha_h = lambda V_m: 0.07*np.exp(-(V_m+65)/20)

Beta_n = lambda V_m: 0.125*np.exp(-(V_m+65)/80)
Beta_m = lambda V_m: 4*np.exp(-(V_m+65)/18)
Beta_h = lambda V_m: 1/(1+np.exp(-(V_m+35)/10))

# Set up constants
Cm = 1
gk = 36
gna = 120
gl = 0.3
Vna = 50
Vk = -77
Vl = -50
V_e = -70

#time step in ms
# I = Cdv/dt + Ik + Ina + Il

# dV/dt = 1/C_m * (I- (Ik + Ina + Il))
Ik = lambda gk, n, V_m, Vk: gk*n**4*(V_m-Vk)
Ina = lambda gna, m, h, V_m, Vna: gna*m**3*h*(V_m-Vna)
Igl = lambda gl, V_m, Vl: gl*(V_m-Vl)

m_inf = Alpha_m(V_e)/(Alpha_m(V_e)+Beta_m(V_e))
h_inf = Alpha_h(V_e)/(Alpha_h(V_e)+Beta_h(V_e))
n_inf = Alpha_n(V_e)/(Alpha_n(V_e)+Beta_n(V_e))

dt = 1e-3
time = np.arange(0,300,dt)

def Vm_eq(X, time):
    V_m, m, h, n = X
    I = 10*((time>100) - (time>150) + (time > 200) - (time>250))
    dVdt = 1/Cm * (I-Ik(gk, n, V_m, Vk) - Ina(gna, m, h, V_m, Vna) - Igl(gl, V_m, Vl))
    dndt = Alpha_n(V_m)*(1-n)-Beta_n(V_m)*n
    dmdt = Alpha_m(V_m)*(1-m)-Beta_m(V_m)*m
    dhdt = Alpha_h(V_m)*(1-h)-Beta_h(V_m)*h
    return dVdt, dmdt, dhdt, dndt

def HH(time):
    # X sets eqn variable, define init V_m, m, h, n
    X = odeint(Vm_eq,  [V_e,  m_inf, h_inf, n_inf], time)
    V = X[:,0]
    return V

plt.plot(time, HH(time))

